In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys

sys.path.append('../resources/library')
sys.path.append('/Users/davide/Developer/ml-tropical-cyclones-detection/resources/library/dynamicopy-0.6.1')
import dynamicopy

import warnings
warnings.filterwarnings('ignore')

Failure in importing the cartopy library, the dynamicopy.cartoplot will not be loaded.     Please install cartopy if you wish to use it.
Failure in importing the cartopy library, the dynamicopy.cartoplot will not be loaded.     Please install cartopy if you wish to use it.


In [2]:
ibtracs_src = '../data/ibtracs/filtered/ibtracs_main-tracks_6h_1980-2021_TS-NR-ET-MX-SS-DS.csv'
owz_src = '/Users/davide/Developer/ml-tropical-cyclones-detection/resources/library/zenodo_bourdin/trackers/OWZ/tracks/ERA5_OWZ.csv'
track_src = '/Users/davide/Developer/ml-tropical-cyclones-detection/resources/library/zenodo_bourdin/trackers/TRACK/ERA5_TRACK.csv'
uz_src = '/Users/davide/Developer/ml-tropical-cyclones-detection/resources/library/zenodo_bourdin/trackers/UZ/tracks/ERA5_UZ.csv'
cnrm_src = '/Users/davide/Developer/ml-tropical-cyclones-detection/resources/library/zenodo_bourdin/trackers/CNRM/tracks_vor15_res5_t1_pt-1_pw5_rel25.csv'

# selected_model = '02_swin_msl_vo_850'
# selected_model = '03_vgg_v3_relu_ks3_msl_vo_850'
# selected_model = '04_vgg_v3_relu_ks5_msl_vo_850'
# selected_model = '05_vgg_v3_linear_ks3_msl_vo_850'
# selected_model = '06_swin_msl_vo_850'
# selected_model = '07_vgg_v3_silu_ks3_msl_vo_850'
# selected_model = '08_vgg_v1_relu_ks3_msl_vo_850'
selected_model = '09_vgg_v1_relu_ks3_msl_vo_850_lnc3'

years = 'all'

our_tracks_src = f'/Users/davide/Developer/ml-tropical-cyclones-detection/data/inference/{selected_model}/tracking.csv'

if years =='test':
    test_years = [1983, 1984, 1993, 1994, 2003, 2004, 2013, 2014]
elif years == 'all':
    test_years = [i for i in range(1980,2020)]
else:
    raise ValueError

lon_range = [100, 320]
lat_range = [0,70]
min_track_count = 12

In [3]:
ibtracs = pd.read_csv(ibtracs_src, index_col=0).rename(columns={'ISO_TIME':'time','LAT':'lat','LON':'lon','SID':'track_id'})[['time','lat','lon','track_id','NATURE','WMO_WIND']]
owz = pd.read_csv(owz_src).rename(columns={' year':'year', ' month':'month', ' day':'day', ' hour':'hour',' lon':'lon',' lat':'lat'})
track = pd.read_csv(track_src, index_col=0)
uz = pd.read_csv(uz_src).rename(columns={' year':'year', ' month':'month', ' day':'day', ' hour':'hour',' lon':'lon',' lat':'lat'})
cnrm = pd.read_csv(cnrm_src).rename(columns={'Longitude':'lon', 'Latitude':'lat', 'Date':'time', 'ID':'track_id'})
ours = pd.read_csv(our_tracks_src, index_col=0).rename(columns={'ISO_TIME':'time', 'LAT':'lat', 'LON':'lon', 'WS':'ws', 'TRACK_ID':'track_id', 'HAVERSINE':'haversine'})

# convert iso timed to pandas datetime format
ibtracs['time'] = pd.to_datetime(ibtracs['time'])
owz['time'] = pd.to_datetime(owz[['year','month','day','hour']])
track['time'] = pd.to_datetime(track['time'])
uz['time'] = pd.to_datetime(uz[['year','month','day','hour']])
cnrm['time'] = pd.to_datetime(cnrm['time'])
ours['time'] = pd.to_datetime(ours['time'])

In [4]:
tmp = pd.merge(left=ours, right=ibtracs, on='time', how='inner')
tmp = tmp[tmp['time'].dt.year.isin(test_years)]
dates = tmp['time'].to_numpy()

In [5]:
# get only observations in dates
ibtracs = ibtracs[ibtracs['time'].isin(dates)].reset_index(drop=True)
owz = owz[owz['time'].isin(dates)].reset_index(drop=True)
track = track[track['time'].isin(dates)].reset_index(drop=True)
uz = uz[uz['time'].isin(dates)].reset_index(drop=True)
cnrm = cnrm[cnrm['time'].isin(dates)].reset_index(drop=True)
ours = ours[ours['time'].isin(dates)].reset_index(drop=True)

# convert longitudes to range [0, 360] format
ibtracs['lon'] = (ibtracs['lon'] + 540) % 360 - 180
owz['lon'] = (owz['lon'] + 540) % 360 - 180
track['lon'] = (track['lon'] + 540) % 360 - 180
uz['lon'] = (uz['lon'] + 540) % 360 - 180
cnrm['lon'] = (cnrm['lon'] + 540) % 360 - 180
ours['lon'] = (ours['lon'] + 540) % 360 - 180

# get only tracks within the lat-lon ranges
ibtracs = ibtracs[(ibtracs['lon']>=lon_range[0]) & (ibtracs['lon']<=lon_range[1]) & (ibtracs['lat']>=lat_range[0]) & (ibtracs['lat']<=lat_range[1])]
owz = owz[(owz['lon']>=lon_range[0]) & (owz['lon']<=lon_range[1]) & (owz['lat']>=lat_range[0]) & (owz['lat']<=lat_range[1])]
track = track[(track['lon']>=lon_range[0]) & (track['lon']<=lon_range[1]) & (track['lat']>=lat_range[0]) & (track['lat']<=lat_range[1])]
uz = uz[(uz['lon']>=lon_range[0]) & (uz['lon']<=lon_range[1]) & (uz['lat']>=lat_range[0]) & (uz['lat']<=lat_range[1])]
cnrm = cnrm[(cnrm['lon']>=lon_range[0]) & (cnrm['lon']<=lon_range[1]) & (cnrm['lat']>=lat_range[0]) & (cnrm['lat']<=lat_range[1])]
ours = ours[(ours['lon']>=lon_range[0]) & (ours['lon']<=lon_range[1]) & (ours['lat']>=lat_range[0]) & (ours['lat']<=lat_range[1])]

# remove too short tracks
ibtracs = ibtracs[ibtracs['track_id'].isin(ibtracs.groupby('track_id').filter(lambda x: len(x) >= min_track_count)['track_id'].unique())].reset_index(drop=True)
owz = owz[owz['track_id'].isin(owz.groupby('track_id').filter(lambda x: len(x) >= min_track_count)['track_id'].unique())].reset_index(drop=True)
track = track[track['track_id'].isin(track.groupby('track_id').filter(lambda x: len(x) >= min_track_count)['track_id'].unique())].reset_index(drop=True)
uz = uz[uz['track_id'].isin(uz.groupby('track_id').filter(lambda x: len(x) >= min_track_count)['track_id'].unique())].reset_index(drop=True)
cnrm = cnrm[cnrm['track_id'].isin(cnrm.groupby('track_id').filter(lambda x: len(x) >= min_track_count)['track_id'].unique())].reset_index(drop=True)
ours = ours[ours['track_id'].isin(ours.groupby('track_id').filter(lambda x: len(x) >= min_track_count)['track_id'].unique())].reset_index(drop=True)

# Track Matching

In [6]:
def compute_pod_and_far(algo, algo_id, obs, max_dist, print_results=False):
    matches = dynamicopy.match_tracks(algo, obs, algo_id, 'ibtracs', max_dist=max_dist, min_overlap=0, ref=True)

    n_gt_match = len(matches[f'id_ibtracs'].unique())
    n_algo_match = len(matches[f'id_{algo_id}'].unique())
    n_observations = len(obs.track_id.unique())
    n_detections = len(algo.track_id.unique())

    H, M, FA = n_gt_match, n_observations - n_gt_match, n_detections - n_algo_match
    POD = H / (H + M)
    FAR = FA / (H + FA)

    if print_results:
        print(f'Algorithm : {algo_id.upper()}\n   Hits : {H}\n   Misses : {M}\n   False Alarms : {FA}\n   POD : {POD}\n   FAR : {FAR}\n')
    return pd.DataFrame(data={
            'algo':[algo_id.upper()], 
            'hits':[H], 
            'misses':[M], 
            'false alarms':[FA], 
            'pod':[POD], 
            'far':[FAR]})

In [7]:
# maximum distance to consider true the match
max_track_distance_matching = 300.0

algo_tracks = {
    'owz': owz, 
    'track': track, 
    'uz': uz, 
    'cnrm': cnrm, 
    'ours': ours, 
}

In [8]:
algo_id = 'owz'
owz_res = compute_pod_and_far(algo_tracks[algo_id], algo_id, ibtracs, max_track_distance_matching, print_results=False)

algo_id = 'track'
track_res = compute_pod_and_far(algo_tracks[algo_id], algo_id, ibtracs, max_track_distance_matching, print_results=False)

algo_id = 'uz'
uz_res = compute_pod_and_far(algo_tracks[algo_id], algo_id, ibtracs, max_track_distance_matching, print_results=False)

algo_id = 'cnrm'
cnrm_res = compute_pod_and_far(algo_tracks[algo_id], algo_id, ibtracs, max_track_distance_matching, print_results=False)

algo_id = 'ours'
ours_res = compute_pod_and_far(algo_tracks[algo_id], algo_id, ibtracs, max_track_distance_matching, print_results=False)

In [9]:
algo_results = pd.concat([owz_res, track_res, uz_res, cnrm_res, ours_res]).reset_index(drop=True)
algo_results

,algo,hits,misses,false alarms,pod,far
0,OWZ,899,279,56,0.763158,0.058639
1,TRACK,1015,163,223,0.861630,0.180129
2,UZ,854,324,11,0.724958,0.012717
3,CNRM,845,333,117,0.717317,0.121622
4,OURS,1017,161,609,0.863328,0.374539


In [10]:
algo_results['years'] = years
algo_results['model'] = selected_model

In [11]:
algo_results['pod'] = algo_results['pod'] * 100
algo_results['far'] = algo_results['far'] * 100

In [12]:
algo_results_src = '/Users/davide/Developer/ml-tropical-cyclones-detection/data/inference/trackers_comparison/algorithms_comparison.csv'
if os.path.exists(algo_results_src):
    algo_results = pd.concat([pd.read_csv(algo_results_src, index_col=0), algo_results]).reset_index(drop=True)
algo_results.to_csv(algo_results_src)

In [13]:
algo_results

,algo,hits,misses,false alarms,pod,far,years,model
0,OWZ,168,59,10,74.008811,5.617978,test,10_swin_lnc3_msl_vo_850
1,TRACK,198,29,45,87.224670,18.518519,test,10_swin_lnc3_msl_vo_850
2,UZ,161,66,2,70.925110,1.226994,test,10_swin_lnc3_msl_vo_850
3,CNRM,158,69,24,69.603524,13.186813,test,10_swin_lnc3_msl_vo_850
4,OURS,178,49,37,78.414097,17.209302,test,10_swin_lnc3_msl_vo_850
...,...,...,...,...,...,...,...,...
95,OWZ,899,279,56,76.315789,5.863874,all,05_vgg_v3_linear_ks3_msl_vo_850
96,TRACK,1015,163,223,86.162988,18.012924,all,05_vgg_v3_linear_ks3_msl_vo_850
97,UZ,854,324,11,72.495756,1.271676,all,05_vgg_v3_linear_ks3_msl_vo_850
98,CNRM,845,333,117,71.731749,12.162162,all,05_vgg_v3_linear_ks3_msl_vo_850
